In [1]:
# !pip install pyro-ppl 

In [1]:
import pandas as pd
from tqdm import tqdm
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pyro
import pyro.distributions as dist
import pyro.infer.elbo
import torch.nn as nn
import torch.nn.functional as F
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.distributions import Normal, Categorical
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import torchvision.models as models

C:\Users\mrper\.conda\envs\ImageGen\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_train_data_info = pd.read_csv("E:/Saumya/RFMiD/Training_Set/RFMiD_Training_Labels.csv")

In [3]:
classes_map = {0: 'Healthy',
 1: 'DR',
 2: 'ARMD',
 3: 'MH',
 4: 'DN',
 5: 'MYA',
 6: 'BRVO',
 7: 'TSLN',
 8: 'ERM',
 9: 'LS',
 10: 'MS',
 11: 'CSR',
 12: 'ODC',
 13: 'CRVO',
 14: 'TV',
 15: 'AH',
 16: 'ODP',
 17: 'ODE',
 18: 'ST',
 19: 'AION',
 20: 'PT',
 21: 'RT',
 22: 'RS',
 23: 'CRS',
 24: 'EDN',
 25: 'RPEC',
 26: 'MHL',
 27: 'RP',
 28: 'OTHER'}

In [4]:
training_image_folder_path = "E:/Saumya/RFMiD/Training_Set/Training/"

In [5]:
imagedata_and_classes_dict = {
    'Image_Path': [],
    'Classes': []
}

In [6]:
for i in tqdm(range(df_train_data_info.shape[0])):
    row = list(df_train_data_info.iloc[i])
    image_name = str(row.pop(0))
    Disease_Risk = row.pop(0)
    if Disease_Risk==0:
        row.insert(0,1)
    else:
        row.insert(0,0)
    image_path = training_image_folder_path+image_name+".png"
    imagedata_and_classes_dict['Image_Path'].append(image_path)
    imagedata_and_classes_dict['Classes'].append(row)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1920/1920 [00:00<00:00, 33673.69it/s]


In [7]:
df_imagedata_and_classes = pd.DataFrame(imagedata_and_classes_dict)

In [8]:
image_transform = transforms.Compose([
    transforms.Resize((512, 340)),  # Resize to a standard size (adjust as needed)
    transforms.ToTensor()
])

In [9]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['Image_Path']
        image = Image.open(img_path).convert('RGB')
        labels = torch.tensor(self.dataframe.iloc[idx]['Classes'], dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, labels

In [10]:
dataset = CustomDataset(df_imagedata_and_classes, transform=image_transform)

In [11]:
Train_Data_Loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [12]:
net = models.inception_v3()

C:\Users\mrper\.conda\envs\ImageGen\Lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [13]:
net.Conv2d_1a_3x3.conv = nn.Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)

In [14]:
net.fc = nn.Linear(in_features=2048, out_features=29, bias=True)

In [15]:
# net.add_module("sigmoid", nn.Sigmoid())

In [16]:
net.aux_logits=False

In [17]:
# net

In [18]:
if torch.cuda.is_available():
    net = net.cuda()

In [28]:
import torch.optim as optim
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

In [29]:
loss_train_list = []
acc_train_list = []

In [21]:
epochs = 5

In [22]:
import numpy as np
min_train_loss = np.inf 

In [23]:
# net.load_state_dict(torch.load("./model_weights/inception/inception_model_weights.pth"))

In [24]:
import time

In [25]:
def convert(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))

In [ ]:
total_time = time.time()
for e in range(epochs):
    start_time=time.time()
    train_loss = 0.0
    right_train = 0
    total_train = 0
    for data, labels in tqdm(Train_Data_Loader):
        # Transfer Data to GPU if available
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
         
        # Clear the gradients
        optimizer.zero_grad()
        net.train()
        # Forward Pass
        target = net(data)
        # Apply sigmoid activation
        sigmoid_output = torch.sigmoid(target)
        # Convert sigmoid output to binary values
        output = (sigmoid_output > 0.5).float()
        # Find the Loss
        loss = criterion(sigmoid_output,labels)
        # Calculate gradients
        loss.backward()
        # Update Weights
        optimizer.step()
        # Calculate Loss
        train_loss += loss.item()
        ftloss = train_loss / len(Train_Data_Loader)
        # Calculate accuracy
        correct_predictions = 1.0 if (output == labels).int().sum()==29 else 0.0
        right_train+=correct_predictions
        total_train+=len(labels)
    ftacc = float(right_train*100/total_train)
    acc_train_list.append(ftacc)
    loss_train_list.append(ftloss)
    print('Epoch',e+1, '\tTraining Loss:',ftloss,"\t time:",convert(time.time()-start_time))
    print("Train Accuracy :",ftacc)
    if min_train_loss > train_loss:
        print("Train Loss Decreased(",min_train_loss,"--->",train_loss,") \t Saving The Model")
        min_train_loss = train_loss
        # Saving State Dict
        torch.save(net.state_dict(), 'E:/Saumya/model_weights/inception/inception_model_weights.pth')
        torch.save(net,"E:/Saumya/model_weights/inception/inception_model.pt")
print("total time : ",convert(time.time()-total_time))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1920/1920 [05:36<00:00,  5.71it/s]


Epoch 1 	Training Loss: 0.1424194855913811 	 time: 00:05:36
Train Accuracy : 10.46875
Train Loss Decreased( 279.3235526215285 ---> 273.4454123354517 ) 	 Saving The Model


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1920/1920 [05:42<00:00,  5.61it/s]


Epoch 2 	Training Loss: 0.13934775361534169 	 time: 00:05:42
Train Accuracy : 11.71875
Train Loss Decreased( 273.4454123354517 ---> 267.54768694145605 ) 	 Saving The Model


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1920/1920 [05:44<00:00,  5.57it/s]


Epoch 3 	Training Loss: 0.13785818008351877 	 time: 00:05:44
Train Accuracy : 12.65625
Train Loss Decreased( 267.54768694145605 ---> 264.68770576035604 ) 	 Saving The Model


  5%|█████▌                                                                                                      | 98/1920 [00:16<05:13,  5.81it/s]

In [27]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

In [59]:
sigmoid_output

tensor([[0.1748, 0.3183, 0.0304, 0.3935, 0.1436, 0.0390, 0.0165, 0.0252, 0.0019,
         0.0333, 0.0042, 0.0295, 0.0160, 0.0087, 0.0025, 0.0082, 0.1089, 0.0387,
         0.0011, 0.0070, 0.0027, 0.0052, 0.0349, 0.0251, 0.0050, 0.0112, 0.0012,
         0.0033, 0.0092]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [62]:
labels

tensor([[0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0')

In [66]:
torch.tanh(target)

tensor([[-0.9141, -0.6421, -0.9980, -0.4075, -0.9453, -0.9967, -0.9994, -0.9987,
         -1.0000, -0.9976, -1.0000, -0.9982, -0.9995, -0.9998, -1.0000, -0.9999,
         -0.9706, -0.9968, -1.0000, -0.9999, -1.0000, -0.9999, -0.9974, -0.9987,
         -1.0000, -0.9997, -1.0000, -1.0000, -0.9998]], device='cuda:0',
       grad_fn=<TanhBackward0>)

In [61]:
criterion(sigmoid_output,labels)

tensor(0.1844, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)

In [42]:
1 if (output == labels).int().sum()==29 else 0

0

In [ ]:
# torch.save(net.state_dict(), '/home/user/research/inception/inception_model_100e_weights.pth')
# torch.save(net,"/home/user/research/inception/inception_model_100e.pt")

In [ ]:
y_v = min(loss_train_list)
x_v = loss_train_list.index(y_v)+1
plt.plot(loss_train_list)
plt.annotate("min train loss",(x_v,y_v))
plt.title('Train Loss during Model Training')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','minimum'], loc='upper left')
plt.show()